In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas
import pandas

In [ ]:
df = pandas.read_csv('/content/drive/MyDrive/diplom/mono_df_all_new.csv', index_col=0, quotechar='&')

df = df[['word', 'POS_ms', 'left_syll', 'syll_onset', 'syll_type', 'right_onset', 'syll_num_reversed', 'left_neigh', 'line_id', 'ict']]

In [ ]:
df['last_syll'] = df.apply(lambda x: (x.syll_num_reversed == 1), axis=1)
df['second_last_syll'] = df.apply(lambda x: (x.syll_num_reversed == 2), axis=1)

In [ ]:
df = df.drop(['syll_num_reversed'], axis=1)

In [ ]:
df = df.fillna('nan')

In [ ]:
df

,word,POS_ms,left_syll,syll_onset,syll_type,right_onset,left_neigh,line_id,ict,last_syll,second_last_syll
0,за,PR,nan,closed,open,open,False,0,False,False,False
1,стол,S,closed,closed,closed,open,False,0,True,False,False
2,я,SPRO,closed,open,open,closed,True,0,False,False,False
3,вновь,ADV,open,closed,closed,closed,True,0,True,False,False
4,об,PR,closed,open,closed,open,False,1,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
3694755,же,PART,open,closed,open,closed,True,2440705,False,False,False
3694756,ты,SPRO,open,closed,open,closed,True,2440705,False,False,False
3694757,где,ADVPRO,nan,closed,open,closed,False,2440706,True,False,False
3694758,же,PART,open,closed,open,closed,True,2440706,False,False,False


In [ ]:
df.to_csv(f'/content/drive/MyDrive/diplom/df_alg.csv', quotechar='&')

In [ ]:
df = pandas.read_csv('/content/drive/MyDrive/diplom/df_alg.csv', index_col=0, quotechar='&')

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=1000, random_state=156)

In [ ]:
sum_data = train.groupby(by=['POS_ms', 'left_syll', 'syll_onset', 'syll_type', 'right_onset', 'left_neigh', 'ict'])['word'].count().to_frame().reset_index()

In [ ]:
sum_data

,POS_ms,left_syll,syll_onset,syll_type,right_onset,left_neigh,ict,word
0,A,closed,closed,closed,closed,False,False,1253
1,A,closed,closed,closed,closed,False,True,2545
2,A,closed,closed,closed,closed,True,False,67
3,A,closed,closed,closed,closed,True,True,1687
4,A,closed,closed,closed,NaN,False,False,24
...,...,...,...,...,...,...,...,...
1220,NaN,open,open,closed,NaN,True,True,8
1221,NaN,open,open,closed,open,False,False,2
1222,NaN,open,open,closed,open,False,True,7
1223,NaN,open,open,closed,open,True,False,2


In [ ]:
sum_data.to_csv(f'/content/drive/MyDrive/diplom/sum_data.csv', quotechar='&')

In [ ]:
sum_data = pandas.read_csv('/content/drive/MyDrive/diplom/sum_data.csv', index_col=0, quotechar='&')

In [ ]:
def is_ict(POS_ms, left_syll, syll_onset, syll_type, right_onset, left_neigh, last_syll, second_last_syll):
    if last_syll == True:
        ict_prob = 0.95
    elif second_last_syll == True:
        ict_prob = 0.07
    else:
        res_df = sum_data[(sum_data['POS_ms'] == POS_ms) &
                          (sum_data['left_syll'] == left_syll) &
                          (sum_data['syll_onset'] == syll_onset) &
                          (sum_data['syll_type'] == syll_type) &
                          (sum_data['right_onset'] == right_onset) &
                          (sum_data['left_neigh'] == left_neigh)]
        if len(res_df) == 0:
            return 0.5
        elif len(res_df) == 1:
            if res_df['ict'].values[0] == True:
                return 1.0
            else:
                return 0.0

        ict_f, ict_t = res_df['word'].values
        ict_prob = round(ict_t/(ict_t + ict_f), 2)
    
    return ict_prob

In [ ]:
test['prob'] = test.apply(lambda x: is_ict(x.POS_ms, x.left_syll, x.syll_onset, x.syll_type, x.right_onset, x.left_neigh, x.last_syll, x.second_last_syll), axis=1)

In [ ]:
#test = test.drop('pred', axis=1)

In [ ]:
test['pred_1'] = test.apply(lambda x: (x.prob > 0.5), axis=1)

In [ ]:
test['res'] = test.apply(lambda x: (x.pred_1 == x.ict), axis=1)

In [ ]:
print(test.groupby(by=['res'])['res'].count())

res
False    193
True     807
Name: res, dtype: int64


In [ ]:
TP = len(test[(test['ict'] == True) & (test['pred_1'] == True)])
FP = len(test[(test['ict'] == False) & (test['pred_1'] == True)])
FN = len(test[(test['ict'] == True) & (test['pred_1'] == False)])
TN = len(test[(test['ict'] == False) & (test['pred_1'] == False)])

In [ ]:
print(f'accuracy: {round((TP + TN)/(TP + TN + FP + FN), 2)}')
print(f'precision: {round(TP/(TP + FP), 2)}')
print(f'recall: {round(TP/(TP + FN), 2)}')

accuracy: 0.81
precision: 0.88
recall: 0.64


In [ ]:
test['pred_2'] = test.apply(lambda x: (x.prob > 0.75), axis=1)

In [ ]:
test['res'] = test.apply(lambda x: (x.pred_2 == x.ict), axis=1)

In [ ]:
print(test.groupby(by=['res'])['res'].count())

res
False    197
True     803
Name: res, dtype: int64


In [ ]:
TP = len(test[(test['ict'] == True) & (test['pred_2'] == True)])
FP = len(test[(test['ict'] == False) & (test['pred_2'] == True)])
FN = len(test[(test['ict'] == True) & (test['pred_2'] == False)])
TN = len(test[(test['ict'] == False) & (test['pred_2'] == False)])

In [ ]:
print(f'accuracy: {round((TP + TN)/(TP + TN + FP + FN), 2)}')
print(f'precision: {round(TP/(TP + FP), 2)}')
print(f'recall: {round(TP/(TP + FN), 2)}')

accuracy: 0.8
precision: 0.94
recall: 0.57


частицы и местоимения

In [ ]:
test[test['res'] == False]

,word,POS_ms,left_syll,syll_onset,syll_type,right_onset,left_neigh,line_id,ict,last_syll,second_last_syll,prob,pred,res
812736,был,V,closed,closed,closed,closed,False,580176,False,False,False,0.75,True,False
3493809,от,PR,closed,open,closed,closed,False,2310173,True,False,False,0.24,False,False
3647815,вот,PART,NaN,closed,closed,closed,False,2405840,True,False,False,0.50,False,False
716435,муж,S,NaN,closed,closed,closed,False,515050,True,False,False,0.50,False,False
1238520,не,PART,closed,closed,open,closed,True,871405,True,False,False,0.22,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2833964,так,ADVPRO,NaN,closed,closed,open,False,1905359,True,False,False,0.50,False,False
3200421,за,PR,closed,closed,open,closed,True,2126783,True,False,False,0.23,False,False
1595610,нем,A,closed,closed,closed,closed,False,1124607,False,False,False,0.67,True,False
38737,что,CONJ,NaN,closed,open,closed,False,31499,True,False,False,0.50,False,False


In [ ]:
test[test['res'] == False].groupby(by=['POS_ms'])['word'].count().sort_values()

POS_ms
INTJ       2
NUM        2
V          8
S         13
APRO      14
ADVPRO    15
PART      27
PR        36
SPRO      38
CONJ      41
Name: word, dtype: int64

In [ ]:
false_functors = test[(test['res'] == False) & ((test['POS_ms'] == 'PR') | (test['POS_ms'] == 'CONJ') | (test['POS_ms'] == 'PART'))]

In [ ]:
verse_data = pandas.read_csv('/content/drive/MyDrive/diplom/syllab-tonic-lines.tsv',sep='\t', quotechar='&')

In [ ]:
for index, row in false_functors.iterrows():
    print(row['word'], '\n', verse_data.loc[row['line_id'], 'line'])

от 
   Бледнѐет пу̀тник о̀т доса̀ды,
вот 
 Во̀т подхо̀дят к са̀мой двѐри,
не 
 Ни в чьѝ глаза̀ -- я зна̀ю -- нѐ глядя̀т.
не 
   Ей нѐ годѝтся ѝ для тѐни
на 
 И по̀местѝлся на̀ складно̀й крова̀ти.
от 
 О̀т меня̀ в обра̀тный пу̀ть.
для 
   Для̀ него̀ прекра̀сен свѐт.
и 
 Кто̀ ещѐ любѝмей ѝ суро̀вей,
но 
 Но̀ во мно̀го ра̀з хитрѐе
о 
 Сквозь да̀же прѐдставлѐнье о̀ себѐ.
что 
 «То̀т Воѐйков, что̀ бранѝлся,
на 
 И на̀ степно̀й просто̀р,
не 
 Да̀ром нѐба нѐ гневѝ!
до 
 И Гѝппокра̀т, встава̀вший до̀ рассвѐта,
и 
 зуба̀ми скрѝпну -- ѝ оста̀нусь.
у 
 У̀ сугро̀бов взбѝты чѐлки
об 
 Мча̀щимися, бря̀кающимися сразма̀ху чуть не  о̀б ноги,
пред 
 Прѐд свято̀й ико̀ной
и 
 ѝ Ван Го̀га за̀ его̀ рома̀шки.
но 
 Но̀ ожерѐлье из слѐз но̀сит Царѝца небѐс.
от 
 Отто̀ржен о̀т тебя̀ безмо̀лвиѐм столѐтий,
на 
 Их пѐрелѝстыва̀ет. / На̀ поля̀х столѐтних
и 
 Ѝ в мину̀ту мра̀ком ту̀ч
а 
 Нѐ житу̀ха, а̀ страда̀
и 
 Крепко-на̀крепко вста̀ли лаба̀зы, обмѐн и обма̀н.
нет 
 нѐт